# Python Profiling and Performance Optimization

## Chapter 8: Robustness and Performance

This notebook covers key concepts from Effective Python regarding profiling and optimizing Python code performance.

---

## Item 70: Profile Before Optimizing

### Key Concepts

The dynamic nature of Python causes surprising performance behaviors:
- Operations you assume are slow may be fast (string manipulation, generators)
- Operations you assume are fast may be slow (attribute access, function calls)
- The true source of slowdowns can be obscure

**Best Practice:** Ignore intuition and directly measure performance before optimizing.

### Example: Profiling an Insertion Sort

Let's profile a simple insertion sort algorithm to identify performance bottlenecks.

In [ ]:
def insertion_sort(data):
    """Sort a list using insertion sort algorithm."""
    result = []
    for value in data:
        insert_value(result, value)
    return result

def insert_value(array, value):
    """Insert value into sorted array (inefficient linear scan version)."""
    for i, existing in enumerate(array):
        if existing > value:
            array.insert(i, value)
            return
    array.append(value)

### Setting Up the Profiler

In [ ]:
from random import randint
from cProfile import Profile
from pstats import Stats

# Create test data
max_size = 10**4
data = [randint(0, max_size) for _ in range(max_size)]
test = lambda: insertion_sort(data)

# Run the profiler
profiler = Profile()
profiler.runcall(test)

### Analyzing Profiler Results

In [ ]:
# Extract and display statistics
stats = Stats(profiler)
stats.strip_dirs()
stats.sort_stats('cumulative')
stats.print_stats()

### Understanding Profiler Columns

- **ncalls**: Number of calls to the function during profiling
- **tottime**: Seconds spent in the function (excluding other function calls)
- **tottime percall**: Average time per call (tottime / ncalls)
- **cumtime**: Cumulative time spent in the function (including other function calls)
- **cumtime percall**: Average cumulative time per call (cumtime / ncalls)

### Optimizing with bisect Module

After profiling reveals `insert_value` is the bottleneck, we can optimize it using binary search:

In [ ]:
from bisect import bisect_left

def insert_value_optimized(array, value):
    """Insert value using binary search (much faster)."""
    i = bisect_left(array, value)
    array.insert(i, value)

# Profile again with optimized version
def insertion_sort_optimized(data):
    result = []
    for value in data:
        insert_value_optimized(result, value)
    return result

test_optimized = lambda: insertion_sort_optimized(data)
profiler_optimized = Profile()
profiler_optimized.runcall(test_optimized)

stats_optimized = Stats(profiler_optimized)
stats_optimized.strip_dirs()
stats_optimized.sort_stats('cumulative')
stats_optimized.print_stats()

### Using print_callers for Call Analysis

When a utility function is called from multiple places, use `print_callers()` to identify which callers contribute most to execution time:

In [ ]:
def my_utility(a, b):
    """Example utility function."""
    c = 1
    for i in range(100):
        c += a * b
    return c

def first_func():
    """Calls my_utility many times."""
    for _ in range(1000):
        my_utility(4, 5)

def second_func():
    """Calls my_utility fewer times."""
    for _ in range(10):
        my_utility(1, 3)

def my_program():
    """Main program calling both functions."""
    for _ in range(20):
        first_func()
        second_func()

# Profile and analyze callers
profiler = Profile()
profiler.runcall(my_program)
stats = Stats(profiler)
stats.print_callers()

---

## Item 71: Prefer deque for Producer-Consumer Queues

### The Problem with list as a FIFO Queue

Using `list.pop(0)` for FIFO queues has **quadratic time complexity** because it requires moving all remaining elements.

In [ ]:
# Example: Email processing queue (problematic implementation)
class Email:
    def __init__(self, sender, receiver, message):
        self.sender = sender
        self.receiver = receiver
        self.message = message

def produce_emails(queue):
    """Producer: adds emails to queue."""
    # Simulated email receiving
    pass

def consume_one_email_slow(queue):
    """Consumer: processes one email (slow with list)."""
    if not queue:
        return
    email = queue.pop(0)  # O(n) operation!
    # Process email...

### Performance Benchmark: list.pop(0)

In [ ]:
import timeit

def print_results(count, tests):
    avg_iteration = sum(tests) / len(tests)
    print(f'Count {count:>5,} takes {avg_iteration:.6f}s')
    return count, avg_iteration

def print_delta(before, after):
    before_count, before_time = before
    after_count, after_time = after
    growth = 1 + (after_count - before_count) / before_count
    slowdown = 1 + (after_time - before_time) / before_time
    print(f'{growth:>4.1f}x data size, {slowdown:>4.1f}x time')

def list_pop_benchmark(count):
    def prepare():
        return list(range(count))
    
    def run(queue):
        while queue:
            queue.pop(0)
    
    tests = timeit.repeat(
        setup='queue = prepare()',
        stmt='run(queue)',
        globals=locals(),
        repeat=1000,
        number=1)
    return print_results(count, tests)

# Run benchmarks
baseline = list_pop_benchmark(500)
for count in (1_000, 2_000, 3_000, 4_000, 5_000):
    comparison = list_pop_benchmark(count)
    print_delta(baseline, comparison)

### Solution: Using collections.deque

The `deque` class provides **constant-time** operations for both ends of the queue.

In [ ]:
import collections

def consume_one_email_fast(queue):
    """Consumer: processes one email (fast with deque)."""
    if not queue:
        return
    email = queue.popleft()  # O(1) operation!
    # Process email...

# Initialize queue with deque
queue = collections.deque()

# Producer still uses append (same as list)
# queue.append(email)

### Performance Benchmark: deque.popleft()

In [ ]:
def deque_popleft_benchmark(count):
    def prepare():
        return collections.deque(range(count))
    
    def run(queue):
        while queue:
            queue.popleft()
    
    tests = timeit.repeat(
        setup='queue = prepare()',
        stmt='run(queue)',
        globals=locals(),
        repeat=1000,
        number=1)
    return print_results(count, tests)

# Run benchmarks
baseline = deque_popleft_benchmark(500)
for count in (1_000, 2_000, 3_000, 4_000, 5_000):
    comparison = deque_popleft_benchmark(count)
    print_delta(baseline, comparison)

---

## Item 72: Consider Searching Sorted Sequences with bisect

### The Problem: Linear Search

Searching for values in a sorted list using `list.index()` takes **linear time** O(n).

In [ ]:
# Linear search example
data = list(range(10**5))
index = data.index(91234)
print(f"Found at index: {index}")

# Finding closest value (linear scan)
def find_closest_linear(sequence, goal):
    """Find index where goal should be inserted (O(n))."""
    for index, value in enumerate(sequence):
        if goal < value:
            return index
    raise ValueError(f'{goal} is out of bounds')

index = find_closest_linear(data, 91234.56)
print(f"Closest index: {index}")

### Solution: Binary Search with bisect

The `bisect` module provides **logarithmic time** O(log n) binary search.

In [ ]:
from bisect import bisect_left

# Exact match
index = bisect_left(data, 91234)
print(f"Exact match at: {index}")
assert index == 91234

# Closest match
index = bisect_left(data, 91234.56)
print(f"Closest match at: {index}")
assert index == 91235

### Performance Comparison

In [ ]:
import random

size = 10**5
iterations = 1000
data = list(range(size))
to_lookup = [random.randint(0, size) for _ in range(iterations)]

def run_linear(data, to_lookup):
    for index in to_lookup:
        data.index(index)

def run_bisect(data, to_lookup):
    for index in to_lookup:
        bisect_left(data, index)

# Benchmark linear search
baseline = timeit.timeit(
    stmt='run_linear(data, to_lookup)',
    globals=globals(),
    number=10)
print(f'Linear search takes {baseline:.6f}s')

# Benchmark bisect
comparison = timeit.timeit(
    stmt='run_bisect(data, to_lookup)',
    globals=globals(),
    number=10)
print(f'Bisect search takes {comparison:.6f}s')

slowdown = 1 + ((baseline - comparison) / comparison)
print(f'{slowdown:.1f}x speedup with bisect')

---

## Item 73: Know How to Use heapq for Priority Queues

### The Problem: Maintaining Priority Order

FIFO queues process items in arrival order, but sometimes you need to process by **priority** (importance).

In [ ]:
# Example: Library book due dates
class Book:
    def __init__(self, title, due_date):
        self.title = title
        self.due_date = due_date

# Naive approach: sort entire list every time (expensive!)
def add_book_slow(queue, book):
    queue.append(book)
    queue.sort(key=lambda x: x.due_date, reverse=True)  # O(n log n)

class NoOverdueBooks(Exception):
    pass

def next_overdue_book_slow(queue, now):
    if queue:
        book = queue[-1]
        if book.due_date < now:
            queue.pop()
            return book
    raise NoOverdueBooks

### Solution: Using heapq Module

A **heap** provides **logarithmic time** O(log n) for insertions and removals.

In [ ]:
from heapq import heappush, heappop, heapify
import functools

# Books must be comparable for heapq
@functools.total_ordering
class Book:
    def __init__(self, title, due_date):
        self.title = title
        self.due_date = due_date
        self.returned = False
    
    def __lt__(self, other):
        return self.due_date < other.due_date
    
    def __repr__(self):
        return f"Book('{self.title}', '{self.due_date}')"

# Fast priority queue operations
def add_book_fast(queue, book):
    heappush(queue, book)  # O(log n)

def next_overdue_book_fast(queue, now):
    while queue:
        book = queue[0]  # Peek at highest priority
        if book.returned:
            heappop(queue)  # Remove returned books
            continue
        if book.due_date < now:
            heappop(queue)  # O(log n)
            return book
        break
    raise NoOverdueBooks

def return_book(queue, book):
    """Mark book as returned (no queue modification needed)."""
    book.returned = True

### Using heapq: Example

In [ ]:
# Create priority queue
queue = []
add_book_fast(queue, Book('Pride and Prejudice', '2019-06-01'))
add_book_fast(queue, Book('The Time Machine', '2019-05-30'))
add_book_fast(queue, Book('Crime and Punishment', '2019-06-06'))
add_book_fast(queue, Book('Wuthering Heights', '2019-06-12'))

print("Books in priority order:")
for book in sorted(queue):
    print(f"  {book}")

# Alternative: create heap from existing list
queue2 = [
    Book('Book A', '2019-06-01'),
    Book('Book B', '2019-05-30'),
    Book('Book C', '2019-06-06'),
]
heapify(queue2)  # O(n) - faster than sorting O(n log n)

# Process overdue books
now = '2019-06-02'
try:
    book = next_overdue_book_fast(queue, now)
    print(f"\nOverdue: {book.title}")
    book = next_overdue_book_fast(queue, now)
    print(f"Overdue: {book.title}")
except NoOverdueBooks:
    print("No more overdue books")

---

## Item 74: Consider memoryview and bytearray for Zero-Copy Interactions

### The Problem: Copying Overhead

Slicing `bytes` creates a **copy** of the data, which can be expensive for large amounts of data.

In [ ]:
# Example: Video streaming server
# Naive approach with copying
video_data = b'x' * (1024 * 1024 * 100)  # 100 MB
byte_offset = 1024 * 1024 * 50
size = 20 * 1024 * 1024  # 20 MB chunk

def send_chunk_with_copy():
    chunk = video_data[byte_offset:byte_offset + size]  # Copies data!
    # socket.send(chunk)
    return chunk

# Benchmark the copy operation
result = timeit.timeit(
    'send_chunk_with_copy()',
    globals=globals(),
    number=100) / 100
print(f'Slice with copy: {result:.9f} seconds')

### Solution: Zero-Copy with memoryview

The `memoryview` type provides access to buffer protocol for **zero-copy** slicing.

In [ ]:
# Zero-copy approach
video_view = memoryview(video_data)

def send_chunk_zero_copy():
    chunk = video_view[byte_offset:byte_offset + size]  # No copy!
    # socket.send(chunk)
    return chunk

# Benchmark zero-copy
result = timeit.timeit(
    'send_chunk_zero_copy()',
    globals=globals(),
    number=100) / 100
print(f'Slice with memoryview: {result:.9f} seconds')

# Inspect memoryview
data = b'shave and a haircut, two bits'
view = memoryview(data)
chunk = view[12:19]
print(f"\nMemoryview: {chunk}")
print(f"Size: {chunk.nbytes}")
print(f"Data in view: {chunk.tobytes()}")
print(f"Underlying data: {chunk.obj}")

### Mutable Buffer Operations with bytearray

For receiving data, use `bytearray` (mutable) with `memoryview` for in-place updates.

In [ ]:
# bytes is immutable
my_bytes = b'hello'
try:
    my_bytes[0] = 0x79
except TypeError as e:
    print(f"bytes error: {e}")

# bytearray is mutable
my_array = bytearray(b'hello')
my_array[0] = 0x79
print(f"\nMutable bytearray: {my_array}")

# memoryview + bytearray for zero-copy writes
my_array = bytearray(b'row, row, row your boat')
my_view = memoryview(my_array)
write_view = my_view[3:13]
write_view[:] = b'-10 bytes-'
print(f"After write: {my_array}")

### Using socket.recv_into for Zero-Copy Receives

In [ ]:
# Example: receiving video stream data
video_cache = b'x' * (1024 * 1024 * 100)  # 100 MB cache
video_array = bytearray(video_cache)
write_view = memoryview(video_array)
byte_offset = 1024 * 1024 * 50
size = 1024 * 1024  # 1 MB chunk

# Slow approach: recv + splicing
def receive_with_copy(socket):
    chunk = socket.recv(size)  # Allocates new bytes
    before = video_view[:byte_offset]
    after = video_view[byte_offset + size:]
    new_cache = b''.join([before, chunk, after])  # Expensive!
    return new_cache

# Fast approach: recv_into with memoryview
def receive_zero_copy(socket):
    chunk = write_view[byte_offset:byte_offset + size]
    socket.recv_into(chunk)  # Writes directly to buffer!
    # No splicing needed

print("Zero-copy receiving provides massive performance improvements!")
print("- Eliminates memory allocation")
print("- Eliminates data copying")
print("- Enables high-throughput I/O operations")

---

## Summary: Performance Best Practices

### Profiling
1. **Always profile before optimizing** - don't trust intuition
2. Use `cProfile` for accurate profiling (not `profile`)
3. Use `pstats.Stats` to analyze profiler output
4. Use `print_callers()` to identify which call sites contribute most

### Data Structures
1. **Use `collections.deque`** for FIFO queues (not `list`)
   - `list.pop(0)` is O(n)
   - `deque.popleft()` is O(1)

2. **Use `bisect`** for searching sorted sequences
   - `list.index()` is O(n)
   - `bisect_left()` is O(log n)

3. **Use `heapq`** for priority queues
   - `list.sort()` is O(n log n)
   - `heappush()`/`heappop()` are O(log n)

### Memory Optimization
1. **Use `memoryview`** for zero-copy slicing of bytes
2. **Use `bytearray`** with `memoryview` for mutable buffers
3. **Use buffer protocol methods** like `socket.recv_into()`

### Key Takeaways
- **Measure, don't guess**: Profile to identify real bottlenecks
- **Choose the right data structure**: Can change O(n²) to O(n log n) or O(n)
- **Avoid unnecessary copies**: Zero-copy operations are dramatically faster
- **Understand complexity**: Know the Big-O behavior of your operations